# Predict NER Model

In [1]:
requirements = [["spacy", "3.0.6"], ["boto3", "1.9.205"], ["json", "1.4.2"]]

In [33]:
import spacy
import os
import boto3
import json
# from dotenv import load_dotenv, find_dotenv #while running locally

In [3]:
# load_dotenv(find_dotenv())

True

In [4]:
my_bucket = "DH-SECURE-OCHATTER"
conn = boto3.client(service_name='s3',
        aws_access_key_id= os.environ.get("s3-access-key"),
        aws_secret_access_key= os.environ.get("s3-secret-key"),
        endpoint_url=os.environ.get("s3-endpoint-url"))

In [5]:
# following commented code is used for saving objects to the bucket and interacting with bucket
# not neccesary in actual workflow

In [6]:
# for key in conn.list_objects(Bucket=my_bucket)['Contents']:
#     print(key['Key'])

In [7]:
# nlp = spacy.load("en_core_web_sm")

In [8]:
# config = nlp.config
# bytes_data = nlp.to_bytes()

In [9]:
# with open('config.json', 'w') as fp:
#     json.dump(config, fp)

# b = open('model_in_bytes', 'wb')
# b.write(bytes_data)
# b.close()

In [10]:
# conn.upload_file(Filename='model_in_bytes',Bucket=my_bucket, Key='model_deploy/model_in_bytes')
# conn.upload_file(Filename='config.json',Bucket=my_bucket, Key='model_deploy/config.json')

In [11]:
obj1 = conn.get_object(Bucket=my_bucket, Key = 'model_deploy/model_in_bytes')

In [12]:
obj1

{'ResponseMetadata': {'RequestId': 'tx00000000000000e3d6727-0060c11cc3-652e5ed3-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '15831120',
   'accept-ranges': 'bytes',
   'last-modified': 'Wed, 09 Jun 2021 19:22:31 GMT',
   'x-rgw-object-type': 'Normal',
   'etag': '"1a191bd2ae70da9ba51204b2b31d7649-2"',
   'x-amz-request-id': 'tx00000000000000e3d6727-0060c11cc3-652e5ed3-default',
   'content-type': 'binary/octet-stream',
   'date': 'Wed, 09 Jun 2021 19:55:47 GMT',
   'connection': 'Keep-Alive',
   'strict-transport-security': 'max-age=31536000; includeSubDomains; preload'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 6, 9, 19, 22, 31, tzinfo=tzutc()),
 'ContentLength': 15831120,
 'ETag': '"1a191bd2ae70da9ba51204b2b31d7649-2"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f1df68b2ef0>}

In [13]:
obj2 = conn.get_object(Bucket=my_bucket, Key = 'model_deploy/config.json')

In [14]:
obj2

{'ResponseMetadata': {'RequestId': 'tx000000000000019be71f0-0060c11cc3-6078b25f-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '5017',
   'accept-ranges': 'bytes',
   'last-modified': 'Wed, 09 Jun 2021 19:22:31 GMT',
   'x-rgw-object-type': 'Normal',
   'etag': '"fecc39263771ebd53d9bd453ff3da01a"',
   'x-amz-request-id': 'tx000000000000019be71f0-0060c11cc3-6078b25f-default',
   'content-type': 'binary/octet-stream',
   'date': 'Wed, 09 Jun 2021 19:55:47 GMT',
   'connection': 'Keep-Alive',
   'strict-transport-security': 'max-age=31536000; includeSubDomains; preload'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 6, 9, 19, 22, 31, tzinfo=tzutc()),
 'ContentLength': 5017,
 'ETag': '"fecc39263771ebd53d9bd453ff3da01a"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f1df6917a58>}

In [15]:
data = obj1['Body'].read()

In [16]:
type(data)

bytes

In [17]:
config_data = obj2['Body'].read()

In [18]:
config_data

b'{"paths": {"train": "corpus/en-core-web/train.spacy", "dev": "corpus/en-core-web/dev.spacy", "vectors": null, "raw": null, "init_tok2vec": null, "vocab_data": null}, "system": {"gpu_allocator": null, "seed": 0}, "nlp": {"lang": "en", "pipeline": ["tok2vec", "tagger", "parser", "senter", "ner", "attribute_ruler", "lemmatizer"], "disabled": ["senter"], "before_creation": null, "after_creation": null, "after_pipeline_creation": null, "batch_size": 256, "tokenizer": {"@tokenizers": "spacy.Tokenizer.v1"}}, "components": {"tok2vec": {"factory": "tok2vec", "model": {"@architectures": "spacy.Tok2Vec.v1", "embed": {"@architectures": "spacy.MultiHashEmbed.v1", "width": "${components.tok2vec.model.encode:width}", "attrs": ["NORM", "PREFIX", "SUFFIX", "SHAPE"], "rows": [5000, 2500, 2500, 2500], "include_static_vectors": false}, "encode": {"@architectures": "spacy.MaxoutWindowEncoder.v1", "width": 96, "depth": 4, "window_size": 1, "maxout_pieces": 3}}}, "tagger": {"factory": "tagger", "model": {"

In [19]:
json_content = json.loads(config_data)

In [20]:
type(json_content)

dict

In [21]:
json_content["nlp"]["lang"]

'en'

In [22]:
lang_cls = spacy.util.get_lang_class(json_content["nlp"]["lang"])
nlpx = lang_cls.from_config(json_content)
nlpx.from_bytes(data)

In [23]:
doc = nlpx("IBM is an organization")
[(ent.text, ent.label_) for ent in doc.ents]

[('IBM', 'ORG')]

## Predictor and Validator

In [24]:
def predictor(feed_text):
    
    doc = nlp(feed_text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [25]:
def validator(x):
    
    return type(x) == str